In [3]:
!pip install openai chromadb python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/4

In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import chromadb

In [6]:
from google.colab import userdata
load_dotenv()

# Get the API key from environment variables
openai_api_key = userdata.get("OPENAI_API_KEY")

In [7]:
# --------------------------------------------------
# 1. ENV + CLIENT SETUP
# --------------------------------------------------
load_dotenv()

client = OpenAI(api_key=openai_api_key)

In [8]:

# --------------------------------------------------
# 2. EMBEDDING FUNCTION
# --------------------------------------------------
def get_embedding(text: str) -> list[float]:
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=text
    )
    return response.data[0].embedding



In [9]:
# --------------------------------------------------
# 3. VECTOR DATABASE SETUP (Chroma)
# --------------------------------------------------
chroma_client = chromadb.Client()

collection = chroma_client.get_or_create_collection(
    name="documents"
)

documents = [
    "LangChain is an orchestration framework for large language models.",
    "RAG stands for Retrieval Augmented Generation.",
    "Vector databases store embeddings for similarity search.",
    "LLMs cannot access private data unless it is provided in the prompt."
]

for idx, doc in enumerate(documents):
    collection.add(
        ids=[str(idx)],
        documents=[doc],
        embeddings=[get_embedding(doc)]
    )

In [11]:
# --------------------------------------------------
# 4. RETRIEVAL FUNCTION
# --------------------------------------------------
def retrieve_context(query: str, k: int = 2) -> str:
    query_embedding = get_embedding(query)

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k
    )

    return "\n".join(results["documents"][0])


# --------------------------------------------------
# 5. LLM ANSWER FUNCTION
# --------------------------------------------------
def answer_question(question: str) -> str:
    context = retrieve_context(question)

    prompt = f"""
You are an expert assistant.
Answer the question using ONLY the context below.

Context:
{context}

Question:
{question}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content


# --------------------------------------------------
# 6. RUN
# --------------------------------------------------
if __name__ == "__main__":
    question = "What is LangChain?"
    answer = answer_question(question)
    print("\nANSWER:\n", answer)



ANSWER:
 LangChain is an orchestration framework for large language models.
